# Functions to apply on the data

In [44]:
# Process the data by day
'''
def preprocess_data_day(data):
    # Convert timestamp to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Group by day
    data = data.groupby([data['timestamp'].dt.date, 'crypto_id']).agg({
        'open': 'first', 
        'close': 'last', 
        'high': 'max', 
        'low': 'min', 
        'volume': 'sum', 
        'number_of_trades': 'sum', 
        'taker_buy_base_asset_volume': 'sum', 
    }).reset_index()

    return data
'''

def preprocess_data_day(data):
    # Convert timestamp to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Group by day
    data = data.groupby([data['timestamp'].dt.date, 'crypto_id']).agg({
        'open': 'first', 
        'close': 'last', 
        'high': 'max', 
        'low': 'min', 
        'volume': 'sum', 
        'quote_asset_volume': 'sum', 
        'number_of_trades': 'sum', 
        'taker_buy_base_asset_volume': 'sum', 
        'taker_buy_quote_asset_volume': 'sum'
    }).reset_index()

    return data

In [45]:
# Apply function to group by month
def group_by_month(df):
    df['date_month'] = df['date'].dt.to_period('M')
    df_month = df.groupby(['date_month', 'crypto_id']).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        #'quote_asset_volume': 'sum',
        'number_of_trades': 'sum',
        #'taker_buy_base_asset_volume': 'sum',
        #'taker_buy_quote_asset_volume': 'sum'
    }).reset_index()
    df_month['diff_oc'] = df_month['close'] - df_month['open']
    df_month['diff_hl'] = df_month['high'] - df_month['low']

    # reagent the columns
    df_month = df_month[['date_month', 'crypto_id', 'open', 'close', 'diff_oc', 'high', 'low', 'diff_hl',
                         'volume', 'number_of_trades']]
    return df_month

In [46]:
# Calculate the difference between open and close and high and low
def calculate_diff(df):
    df['diff_oc'] = df['close'] - df['open']
    df['diff_hl'] = df['high'] - df['low']
    return df

In [47]:
# Calcualte the means of open and close and high and low
def calculate_means(df):
    df['mean_oc'] = (df['open'] + df['close']) / 2
    df['mean_hl'] = (df['high'] + df['low']) / 2
    return df

In [48]:
# Create a dataframe for each year with 'date_month' separed by months
def create_dataframes_by_year(df, years):
    return {year: df[df['date_month'].dt.year == year] for year in years}

In [49]:
# set index 'date' and 'crypto_id' 
def set_index_and_display(data, index_columns, num_rows=30):
    data.set_index(index_columns, inplace=True)
    return data.head(num_rows)

In [50]:
# Transfor the columns to percentage
def transform_to_percentage(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: f'{x:.2f}%')
    return df

In [51]:
# For the graphic, function to calculate the percentage change, by year
def calculate_percentage_change(data):
    data = data.sort_values(by=['date_month'])
    data = data.groupby('id_name').agg({'open': 'first', 'close': 'last'})
    data['change'] = ((data['close'] - data['open']) / data['open']) * 100
    return data

In [52]:
# Function to plot the percentage change by year
def plot_percentage_change(data, year):
    plt.figure(figsize=(12, 8))
    plt.bar(data.index, data['change'], color='lightblue')
    plt.xlabel('Cryptocurrency')
    plt.ylabel('Percentage Change')
    plt.title(f'Cryptocurrency Percentage Change in {year}')
    plt.xticks(rotation=40)
    for i, v in enumerate(data['change']):
        plt.text(i, v, str(round(v, 2)) + '%', ha='center', va='bottom')
    plt.show()

In [53]:
# use time_periods on the function plot_number_of_trades_for_year
def plot_number_of_trades_for_year(year, data):
    """
    Plots the number of trades for each crypto_id and month for a given year.

    Parameters:
    year (int): The year to plot data for.
    data (DataFrame): The data to plot.
    """
    data_trade = data.loc[data['date_month'].dt.year == year]
    data_trade['month'] = data_trade['date_month'].dt.strftime('%b')
    data_trade['month'] = pd.Categorical(data_trade['month'], categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)
    data_trade = data_trade.groupby(['month', 'id_name'])['number_of_trades'].sum().unstack().fillna(0)
    
    ax = data_trade.plot(kind='barh', stacked=True, figsize=(10, 8), color=[color_dict.get(x, '#333333') for x in data_trade.columns])
    plt.title(f'Number of Trades for Cryptocurrencies in {year}')
    plt.xlabel('Number of Trades')
    plt.ylabel('Month')
    plt.legend(title='Cryptocurrency', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

In [54]:
# Function to plot the top 5 performing cryptocurrencies by year

def plot_open_close_for_crypto(data, crypto_id, title):
    # Filter the data for the given crypto_id
    crypto_data = data.loc[data['crypto_id'] == crypto_id]
    crypto_data = crypto_data[['open', 'close']]
    crypto_data = crypto_data.set_index(data['date_month'])

    # Plot the open and close prices
    crypto_data.plot(figsize=(10, 6))
    plt.title(f'Open and Close for {title} from 2020 to 2024')
    plt.ylabel('Price in USD')

    # Add value labels to the graph only at the January lines
    for date, open_val, close_val in zip(crypto_data.index, crypto_data['open'], crypto_data['close']):
        if date.month == 1:  # Check if the month is January
            plt.text(date.to_timestamp(), open_val, f'{open_val:.2f}', ha='right', va='bottom', fontsize=8, color='blue')
            plt.text(date.to_timestamp(), close_val, f'{close_val:.2f}', ha='right', va='bottom', fontsize=8, color='red')

    plt.show()